In [1]:
!pip install openai==0.28.0

In [2]:
!pip install --upgrade openai

  Using cached openai-1.56.1-py3-none-any.whl.metadata (24 kB)
Using cached openai-1.56.1-py3-none-any.whl (389 kB)
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0


In [7]:
from dotenv import load_dotenv
import os
import pandas as pd
import re
import openai
import difflib
from openai import OpenAI


class ASUSLaptopRecommendationChatbot:
    def __init__(self, csv_file, openai_api_key):
        # Set up OpenAI API
        openai.api_key = openai_api_key

        # Read the CSV file
        self.df = pd.read_csv(csv_file, encoding='utf-8')

        # Clean and preprocess the data
        self.df['NamaBarang'] = self.df['NamaBarang'].fillna('')

        # Remove the sales information from the product name
        self.df['CleanName'] = self.df['NamaBarang'].apply(self._clean_product_name)

        # Create a dictionary to track product sales
        self.sales_dict = self._get_sales_dict()

        # Prepare initial system context
        self.conversation_history = [
            {"role": "system", "content": """
            Anda adalah asisten penjualan profesional untuk laptop ASUS.
            Tugas Anda adalah membantu pelanggan menemukan laptop ASUS yang tepat
            sesuai dengan kebutuhan mereka. Gunakan data katalog yang tersedia untuk
            memberikan rekomendasi yang akurat dan membantu pelanggan memilih laptop
            yang sesuai dengan anggaran dan keperluannya.

            Beberapa kategori laptop ASUS yang tersedia:
            - VivoBook (untuk penggunaan umum)
            - ROG (Republic of Gamers, untuk gaming)
            - ZenBook (untuk profesional)
            - TUF Gaming (gaming dengan harga terjangkau)

            Selalu tanyakan kebutuhan spesifik pelanggan seperti:
            - Tujuan penggunaan (gaming, kerja, desain, dll)
            - Anggaran
            - Preferensi ukuran layar
            - Kebutuhan kinerja
            """}
        ]

    def _clean_product_name(self, name):
        """Clean product name by removing extra details and sales info."""
        # Remove text in square brackets
        name = re.sub(r'\[.*?\]', '', name)
        # Remove specific sales-related text
        name = name.replace('NO BAG', '').replace('TRADE IN', '').strip()
        return name

    def _get_sales_dict(self):
        """Create a dictionary tracking sales count for each product."""
        sales_dict = {}
        for index, row in self.df.iterrows():
            # Convert sales to numeric, handling '80+', '2rb+' type of entries
            sales = row['terjual']
            if isinstance(sales, str):
                sales = sales.replace('+', '').replace('rb', '000')

            try:
                sales_count = int(sales)
                sales_dict[row['CleanName']] = sales_count
            except (ValueError, TypeError):
                sales_dict[row['CleanName']] = 0

        return sales_dict

    def find_matching_laptops(self, query, max_recommendations=5):
        """Find laptop matches based on query."""
        # Find close matches to the query
        matches = difflib.get_close_matches(query, self.df['CleanName'], n=max_recommendations, cutoff=0.1)

        if not matches:
            # If no direct matches, try partial matching
            matches = [name for name in self.df['CleanName'] if query.lower() in name.lower()]

        # Prepare recommendation details
        recommendations = []
        for match in matches:
            # Find the full product details
            product_details = self.df[self.df['CleanName'] == match].iloc[0]
            recommendations.append({
                'name': product_details['NamaBarang'],
                'price': product_details['Harga'],
                'sales': product_details['terjual']
            })

        return recommendations

    def get_gpt_response(self, user_input):
        """Generate response using GPT and integrate product recommendations."""
        # Find matching laptops
        matching_laptops = self.find_matching_laptops(user_input)

        # Prepare laptop recommendations for GPT context
        laptop_context = "Laptop yang mungkin sesuai:\n"
        for laptop in matching_laptops:
            laptop_context += f"- {laptop['name']} (Harga: {laptop['price']}, Terjual: {laptop['sales']})\n"

        # Add user input to conversation history
        self.conversation_history.append({"role": "user", "content": user_input + "\n\n" + laptop_context})

        # Generate response using GPT
        try:
            client=OpenAI()
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=self.conversation_history,
                max_tokens=150,
                temperature=0.7
            )

            # Extract and return the assistant's response
            gpt_response = response.choices[0].message.content

            # Add to conversation history
            self.conversation_history.append({"role": "assistant", "content": gpt_response})

            return gpt_response

        except Exception as e:
            return f"Maaf, terjadi kesalahan: {str(e)}"

    def start_chat(self):
        """Start an interactive chat session."""
        print("Selamat datang di Asisten Rekomendasi Laptop ASUS!")
        print("Ketik 'keluar' untuk mengakhiri percakapan.")

        while True:
            try:
                query = input("\nAda yang bisa saya bantu? ")

                if query.lower() in ['keluar', 'exit', 'quit']:
                    print("Terima kasih telah menggunakan layanan kami. Sampai jumpa!")
                    break

                response = self.get_gpt_response(query)
                print("\nAsisten ASUS:", response)

            except KeyboardInterrupt:
                print("\nPercakapan diakhiri.")
                break

def main():
    # Dapatkan API key dari environment variable
    load_dotenv()
    openai_api_key = os.getenv('OPENAI_API_KEY')

    if not openai_api_key:
        print("Silakan atur OPENAI_API_KEY terlebih dahulu di environment variable.")
        return
    
    # Path to your CSV file
    csv_file = 'Dataset/asus.csv'

    # Create chatbot instance
    chatbot = ASUSLaptopRecommendationChatbot(csv_file, openai_api_key)

    # Start the chat
    chatbot.start_chat()

if __name__ == "__main__":
    main()

Selamat datang di Asisten Rekomendasi Laptop ASUS!
Ketik 'keluar' untuk mengakhiri percakapan.



Ada yang bisa saya bantu?  laptop dibawah 5juta



Asisten ASUS: Berdasarkan anggaran di bawah 5 juta, laptop ASUS Vivobook Go 14 E410KA-FHD425 - Star Black bisa menjadi pilihan yang bagus. Laptop ini memiliki spesifikasi sebagai berikut:
- Prosesor: Intel Celeron N4500
- Grafis: Intel UHD Graphics
- RAM: 4GB
- Storage: 256GB SSD
- Layar: 14 inch
- Sistem Operasi: Windows 11
- Office: Microsoft Office Home & Student

Laptop ini cocok untuk penggunaan sehari-hari seperti browsing, menonton video, dan bekerja dengan aplikasi standar. Meskipun spesifikas



Ada yang bisa saya bantu?  selain itu?



Asisten ASUS: Untuk laptop ASUS dengan anggaran di bawah 5 juta, Anda juga bisa mempertimbangkan ASUS Vivobook 14 A1404ZA-VIPS354 - Quiet Blue. Laptop ini memiliki spesifikasi sebagai berikut:
- Prosesor: Intel Core i3-1215U
- Grafis: Intel UHD Graphics
- RAM: 8GB
- Storage: 512GB SSD
- Layar: 14 inch
- Sistem Operasi: Windows 11
- Office: Microsoft Office Home & Student

Laptop ini cocok untuk kebutuhan sehari-hari seperti bekerja, belajar, dan hiburan ringan. Dengan RAM yang lebih besar, laptop ini


...


Asisten ASUS: Maaf, saat ini tidak ada laptop ASUS yang tersedia secara gratis. Jika Anda memiliki pertanyaan lain atau memerlukan bantuan dalam memilih laptop ASUS yang sesuai dengan kebutuhan dan anggaran Anda, jangan ragu untuk bertanya. Saya siap membantu Anda.



Ada yang bisa saya bantu?  bisa nyicil gk?



Asisten ASUS: Untuk informasi tentang kemungkinan program pembiayaan atau kredit untuk pembelian laptop ASUS, Anda bisa langsung menghubungi toko atau penyedia layanan pembiayaan yang menjual produk tersebut. Mereka akan memberikan informasi lengkap mengenai opsi pembiayaan yang tersedia, termasuk kemungkinan untuk melakukan cicilan.

Jika Anda memiliki pertanyaan lain atau butuh bantuan lebih lanjut, jangan ragu untuk bertanya. Saya siap membantu Anda.



Ada yang bisa saya bantu?  exit


Terima kasih telah menggunakan layanan kami. Sampai jumpa!


In [8]:
import json

class ASUSLaptopRecommendationChatbot:
    def __init__(self, csv_file, openai_api_key, history_file="conversation_history.json"):
        openai.api_key = openai_api_key

        # Read the CSV file
        self.df = pd.read_csv(csv_file, encoding='utf-8')
        self.df['NamaBarang'] = self.df['NamaBarang'].fillna('')
        self.df['CleanName'] = self.df['NamaBarang'].apply(self._clean_product_name)
        self.sales_dict = self._get_sales_dict()

        self.history_file = history_file
        self.conversation_history = self._load_history()

        if not self.conversation_history:  # If no previous history, initialize system context
            self.conversation_history = [
                {"role": "system", "content": """
                Anda adalah asisten penjualan profesional untuk laptop ASUS.
                Tugas Anda adalah membantu pelanggan menemukan laptop ASUS yang tepat
                sesuai dengan kebutuhan mereka. Gunakan data katalog yang tersedia untuk
                memberikan rekomendasi yang akurat dan membantu pelanggan memilih laptop
                yang sesuai dengan anggaran dan keperluannya.

                Beberapa kategori laptop ASUS yang tersedia:
                - VivoBook (untuk penggunaan umum)
                - ROG (Republic of Gamers, untuk gaming)
                - ZenBook (untuk profesional)
                - TUF Gaming (gaming dengan harga terjangkau)

                Selalu tanyakan kebutuhan spesifik pelanggan seperti:
                - Tujuan penggunaan (gaming, kerja, desain, dll)
                - Anggaran
                - Preferensi ukuran layar
                - Kebutuhan kinerja
                """}
            ]

    def _load_history(self):
        """Load conversation history from a file."""
        try:
            with open(self.history_file, 'r', encoding='utf-8') as file:
                return json.load(file)
        except FileNotFoundError:
            return []

    def _save_history(self):
        """Save conversation history to a file."""
        with open(self.history_file, 'w', encoding='utf-8') as file:
            json.dump(self.conversation_history, file, ensure_ascii=False, indent=4)

    def get_gpt_response(self, user_input):
        """Generate response using GPT and integrate product recommendations."""
        matching_laptops = self.find_matching_laptops(user_input)
        laptop_context = "Laptop yang mungkin sesuai:\n"
        for laptop in matching_laptops:
            laptop_context += f"- {laptop['name']} (Harga: {laptop['price']}, Terjual: {laptop['sales']})\n"

        self.conversation_history.append({"role": "user", "content": user_input + "\n\n" + laptop_context})

        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=self.conversation_history,
                max_tokens=150,
                temperature=0.7
            )

            gpt_response = response.choices[0].message.content
            self.conversation_history.append({"role": "assistant", "content": gpt_response})
            self._save_history()  # Save updated history to file

            return gpt_response

        except Exception as e:
            return f"Maaf, terjadi kesalahan: {str(e)}"

    def start_chat(self):
        print("Selamat datang di Asisten Rekomendasi Laptop ASUS!")
        print("Ketik 'keluar' untuk mengakhiri percakapan.")

        while True:
            try:
                query = input("\nAda yang bisa saya bantu? ")

                if query.lower() in ['keluar', 'exit', 'quit']:
                    print("Terima kasih telah menggunakan layanan kami. Sampai jumpa!")
                    break

                response = self.get_gpt_response(query)
                print("\nAsisten ASUS:", response)

            except KeyboardInterrupt:
                print("\nPercakapan diakhiri.")
                break


AttributeError: 'ASUSLaptopRecommendationChatbot' object has no attribute '_clean_product_name'